# Team Project Spotify Popular Song
> Date: 09/24/2023
> Group 7: Arya Shahbazi and Ravita Kartawinata



### About:
Project: Predict Song Popularity 
Background: The dataset consists of insightful song attributes such as artists, song features, release date and chart from other platforms which would be used to predict a song’s success or popularity. The features can be used to identify pattern of popular audio features or identify song’s popularity across the platforms or identify artists involvement to the success of the song.

### Library and Dataset

In [1]:
import pandas as pd
import numpy as np
from statistics import mode, mean
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler


from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_curve, auc, accuracy_score, recall_score, precision_score


import matplotlib.pylab as plt

from dmba import plotDecisionTree, gainsChart, liftChart
from dmba import classificationSummary, regressionSummary

%matplotlib inline

In [2]:
# Load the data
data = pd.read_csv("C:/Users/rkartawi/Desktop/Ravita/MSADS/505/Project/spotify-2023.csv", encoding='ISO-8859-1')
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 953 entries, 0 to 952
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   track_name            953 non-null    object
 1   artist(s)_name        953 non-null    object
 2   artist_count          953 non-null    int64 
 3   released_year         953 non-null    int64 
 4   released_month        953 non-null    int64 
 5   released_day          953 non-null    int64 
 6   in_spotify_playlists  953 non-null    int64 
 7   in_spotify_charts     953 non-null    int64 
 8   streams               953 non-null    object
 9   in_apple_playlists    953 non-null    int64 
 10  in_apple_charts       953 non-null    int64 
 11  in_deezer_playlists   953 non-null    object
 12  in_deezer_charts      953 non-null    int64 
 13  in_shazam_charts      903 non-null    object
 14  bpm                   953 non-null    int64 
 15  key                   858 non-null    ob

### Data Preprocessing

In [3]:
# change data type to numeric
data['streams'] = pd.to_numeric(data['streams'], errors='coerce')
data['in_deezer_playlists'] = pd.to_numeric(data['in_deezer_playlists'], errors='coerce')
data['in_shazam_charts'] = pd.to_numeric(data['in_shazam_charts'], errors='coerce')


In [4]:
#Check Missing Values

for col in data.columns:
    count = data[col].isnull().sum()
    if count > 0:
        print(f"Column: {col}, Count of 'Null': {count}")


Column: streams, Count of 'Null': 1
Column: in_deezer_playlists, Count of 'Null': 79
Column: in_shazam_charts, Count of 'Null': 57
Column: key, Count of 'Null': 95


In [5]:
#replace key null value with mode and in_shazam_charts with mean
data['key'].fillna(mode(data['key']), inplace=True)
data['in_shazam_charts'].fillna(data['in_shazam_charts'].mean(), inplace=True)


In [ ]:
# bin similiar columns
tune = ['bpm', 'danceability_%', 'energy_%', 'valence_%', 'acousticness_%', 
                  'instrumentalness_%', 'liveness_%', 'speechiness_%']

list_chart = ['in_spotify_playlists', 'in_spotify_charts', 'in_apple_playlists', 
              'in_apple_charts', 'in_deezer_playlists', 'in_deezer_charts',
              'in_shazam_charts']


### Data Visualization